In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from PIL import Image
import os
import datetime
import csv
from slackclient import SlackClient
from env import *

def slack_message(message, channel):
    token = SLACK_TOKEN
    sc = SlackClient(token)
    sc.api_call('chat.postMessage', channel=channel, 
                text=message, username='queue_update')

def val2key(d, search_v):
    for k,v in d.iteritems():
        if v == search_v:
            return k

driver = webdriver.Chrome('/Users/jskycak/automation/chromedriver_mac')
driver.get('https://cms8.remilon.com/login')

username = driver.find_element_by_name("username")
password = driver.find_element_by_name("password")

username.send_keys("jskycak")
password.send_keys(CMS_PASS)

driver.find_element_by_class_name("btn").click()

previous_num_items_available = 'XXX'
while True:
    driver.get('https://cms8.remilon.com/work#!/available')
    time.sleep(60)
    screenshot = driver.save_screenshot('temp/available_items_screenshot.png')
    screenshot_time = datetime.datetime.now()
    print '------------------------\n[BEGIN]',screenshot_time
    
    #previous_tens = Image.open('temp/available_items_template_tens_previous.png')
    
    img = Image.open('temp/available_items_screenshot.png')
    width, height = img.size[0], img.size[1]
    area_full = (int(0.761*width), int(0.27*height), int(0.78*width), int(0.32*height))
    area_hundreds = (int(0.761*width), int(0.27*height), int(0.768*width), int(0.32*height))
    area_tens = (int(0.767*width), int(0.27*height), int(0.774*width), int(0.32*height))
    area_ones = (int(0.773*width), int(0.27*height), int(0.78*width), int(0.32*height))
    
    img.crop(area_full).save('temp/available_items_template_full.png')
    img.crop(area_ones).save('temp/available_items_template_ones.png')
    img.crop(area_tens).save('temp/available_items_template_tens.png')
    img.crop(area_hundreds).save('temp/available_items_template_hundreds.png')
    
    existing_templates_full = {filename: Image.open('templates/full/'+filename) for filename in os.listdir('templates/full')}
    existing_templates_ones = {filename: Image.open('templates/ones/'+filename) for filename in os.listdir('templates/ones')}
    existing_templates_tens = {filename: Image.open('templates/tens/'+filename) for filename in os.listdir('templates/tens')}
    existing_templates_hundreds = {filename: Image.open('templates/hundreds/'+filename) for filename in os.listdir('templates/hundreds')}

    template_full = Image.open('temp/available_items_template_full.png')
    template_ones = Image.open('temp/available_items_template_ones.png')
    template_tens = Image.open('temp/available_items_template_tens.png')
    template_hundreds = Image.open('temp/available_items_template_hundreds.png')
    
    ones_str = 'X'
    tens_str = 'X'
    hundreds_str = 'X'
    save_filename = screenshot_time.strftime("%Y_%m_%d_%H_%M_%S")+".png"
    if template_full not in existing_templates_full.values():
        template_full.save("templates/full/"+save_filename)
        
        if template_ones not in existing_templates_ones.values():
            template_ones.save("templates/ones/"+save_filename)
            print "saved templates/ones/"+save_filename
        else:
            ones_str = val2key(existing_template_ones, template_ones)[0]
            print 'found ones = '+ones_str
            
        if template_tens not in existing_templates_tens.values():
            template_tens.save("templates/tens/"+save_filename)
            print "saved templates/tens/"+save_filename
        else:
            tens_str = val2key(existing_template_tens, template_tens)[0]
            print 'found tens = '+tens_str
            
        if template_hundreds not in existing_templates_hundreds.values():
            template_hundreds.save("templates/hundreds/"+save_filename)
            print "saved templates/hundreds/"+save_filename
        else:
            hundreds_str = val2key(existing_template_hundreds, template_hundreds)[0]
            print 'found hundreds = '+hundreds_str
            
        num_items_available = hundreds_str + tens_str + ones_str
        
    else:
        num_items_available = val2key(existing_templates_full, template_full)[0:3]
        print 'found num_items_available = '+num_items_available
    
    with open('queue_log.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerow([screenshot_time, num_items_available])
    
    try:
        num_items_delta = str(int(num_items_available) - int(previous_num_items_available))
        if int(num_items_delta) > 0:
            slack_message('{0} items added ({1} available)'.format(*[num_items_delta, num_items_available]),'queue_update')
            print '[END] {0} | {1} | +{2}'.format(*[screenshot_time, num_items_available, num_items_delta])
        else: 
            print '[END] {0} | {1} | {2}'.format(*[screenshot_time, num_items_available, num_items_delta])
    except:
        print '[END] {0} | {1} | n/a'.format(*[screenshot_time, num_items_available])
    
    previous_num_items_available = num_items_available
    # if tens place changed since last time, send me an alert
    # figure out how to send image in channel?
    #if template_tens != previous_tens:
    #    slack_message('Queue 10s place has changed','queue_update')
    #template_tens.save('temp/available_items_template_tens_previous.png')

driver.quit()

In [ ]:
!rm templates/*/.DS_store